How to turn this notebook into slides? Run:
```
$ ipython nbconvert --to slides lecture.ipynb  --post serve
```
OR (better), install the RISE Jupyter extension:
```
conda install -c damianavila82 rise
```
and hit the 'Slideshow' toolbar button.

This notebook uses the [python-markdown](http://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/python-markdown/readme.html) extension, allowing it to run code inline with Markdown text.

To install this extension, and the [configurator GUI](https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator), while Jupyter is ***not*** running run:
```bash
conda install -c defaults -c conda-forge jupyter_contrib_nbextensions
conda install -c defaults -c conda-forge jupyter_nbextensions_configurator
```
and then enable python-markdown in the `Nbextensions` tab that will show up next time you start Jupyter.

To have the markup in all cells show up in this notebook, you need to mark it as 'Trusted'. The notebook will be marked as trusted once you re-evaluate all cells.

Some helper functions to make it easier to show code listings.

In [1]:
def show_code_listing(fn):
    from IPython.display import display, Markdown
    return Markdown( '```python\n{}\n```'.format(open(fn).read()))

def execute_and_show(cmdline):
    from IPython.display import display, Markdown
    res = ! $cmdline
    res = '\n'.join(['    ' + line for line in res])
    print('```bash\n    ${}\n{}\n```'.format(cmdline, res))

## Problem 5: A Better Way to Deal with Errors -- Exceptions

Note how most of our code now is just error checking and handling. The actual purpose of our code is completely hidden by all the error checking code!

This doesn't have to be the case. In Python, it is possible to elegantly separate the error checking code from the main algorithm through the use of [exceptions](https://docs.python.org/3/tutorial/errors.html#exceptions). The big idea is that code can _raise an exception_ when some assumption or rule is violated.

Example:

In [2]:
def div(a, b):
    return a / b

#print("Division result: {}".format(div(1, 2)))
print("Division result: {}".format(div(1, 0)))

ZeroDivisionError: division by zero

When an exception is raised, Python stops executing the program and returns immediately from the called function. If an exception is not *caught*, it will interupt the program and print out a _stack trace_ (the output above) that gives you an idea of where an exception occured.

You've encountered this already:
{{ execute_and_show('./prog3.py 2 x') }}
The output above is the result of an _uncaught exception_.

The idea is that you can "catch" these exceptions, and handle the error condition gracefully (i.e., by writing out a nice error message). This is accomplished through the use of a "`try ... except`" clause:

In [3]:
def div(a, b):
    return a / b

try:
    print("Division result: {}".format(div(1, 2)))
    print("Division result: {}".format(div(1, 0)))
    print("Division result: {}".format(div(1, 3)))
except ZeroDivisionError as err:
    print(err)
    
print("I'm not dead!")

Division result: 0.5
division by zero
I'm not dead!


What's happened:
* We've now wrapped our calls to `div()` in a `try...except` block.
* When we attempted to do something "illegal" (divide by zero), an exception was raised nd the program immediately jumped to the first `except` block that matched the type of the raised exception. We say that an exception has been _handled_.
* After that, execution continued (note how it printed the message about not being dead)
* Finally, note how `div(1, 3)` line has not been invoked -- because an exception has been raised, the execution continued in the `except` block. Python does not "go back" after handling the exception!

### Exceptions are the "Pythonic Way"

This is the preferred way to signal and check for errors in Python. Virtually all errors ("exceptional situations") are reported by raising an exception.

Here's another example -- indexing past the end of a list:

In [4]:
a = ["a", "b", "c"]
a[4]

IndexError: list index out of range

We can have multiple `except` blocks, to handle different kinds of exceptions:

In [5]:
arr = [0, 1, 2, 3]
i, j = 2, 0
try:

    print(arr[i] / arr[j])

except ZeroDivisionError as err:
    print(err)
except IndexError as err:
    print(err)
    
print("moving on!")

division by zero
moving on!


Compare that to:

In [6]:
arr = [0, 1, 2, 3]
i, j = 1, 5

if i < 0 or i >= len(arr) or j < 0 or j >= len(arr):
    print("list index out of range")
elif arr[j] == 0:
    print("division by zero")
else:
    print(arr[i] / arr[j])

print("moving on!")

list index out of range
moving on!


Note:
* The code using exceptions is _cleaner_ -- it neatly separates the part that does work (divides two numbers) from the part that handles exceptional situations.
* The code using exceptions is _shorter_ -- shorter coe, fewer bugs.
* The philosophical approach to errors is fundamentally different:
  1. the implementation with `if` statements tries to _avoid_ even trying to break the rules (dividing by zero, indexing out of bounds, ...)
  1. the implementation with exceptions detects when the rules were attempted to be broken and reacts to that situation.

## It's Easier to Ask for Forgiveness than Permission

What we've done above is an example of the [***EAFP***](https://docs.python.org/3.6/glossary.html) coding style:

>   It's **E**asier to **A**sk for **F**orgiveness, than **P**ermission.
>

that is core to well-written, readable, Python code.

How do you know what exceptions you could (should) be catching?
* The documentation of (typically) tells you which exceptions any given function may raise.
* Look at Python's [list of built-in exceptions](https://docs.python.org/3/library/exceptions.html)
* Try it!

## You Can Also Raise Exceptions Explicitly

In [7]:
def spectral_type_temperature(spectype):

    _type_to_temp = dict(
        O=54_000,
        B=29_200,
        A=9600,
        F=7350,
        G=6050,
        K=5240,
        M=3750
    )

    if spectype not in _type_to_temp:
        raise Exception(f"Unknown spectral type '{spectype}'")

    return _type_to_temp[spectype]

spectral_type_temperature("X")

Exception: Unknown spectral type 'X'

## Group Work

Now back to our program (`prog5.py`): let's change it so that instead of the many if statements, we use exceptions.

Hints:
* You should `try` to access and convert the command line arguments to floating point numbers and use `except` to catch any errors and display error messages.
* The `number_or_exit` function should go away.

## Implementation

In [10]:
show_code_listing("prog8.py")

FileNotFoundError: [Errno 2] No such file or directory: 'prog8.py'

Discusion:
* This is _much_ cleaner now! Just a quick glance gives you an idea what this code does!
* Exceptions are "caught" in a [`try-except` block](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).
* There was no need for the `number_or_exit()` function any more, or the messy loop, etc.!
* The code is now much more [_Pythonic_](http://docs.python-guide.org/en/latest/writing/style/).

Remember:

>   It's **E**asier to **A**sk for **F**orgiveness, than **P**ermission.
>

This is the Pythonic Way!

## Problem 6: Product of all arguments

Now let's extend the program to compute the _product_ of all arguments given on the command line. Example:
{{ execute_and_show('./prog9.py 1.5 2.5 3.5 4.5') }}

## Implementation

In [11]:
show_code_listing("prog9.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

#### The main program begins here

try:
    prod = 1
    for arg in sys.argv[1:]:
        prod *= float(arg)
    print(prod)
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

Discusion:
* The change was pretty straightforward; just a `for`-loop over all arguments
* No need to catch the `IndexError` exception any more

## Implementation v2

Let's take a slightly different approach. Note we can view this as a two-part problem:

1. Converting the list of command line arguments from strings to floats
1. Summing up the resulting list.

So another solution could be:

In [12]:
show_code_listing("prog10.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

def to_floats(lst):
    """ Returns a list of strings corresponding to a list of floats """
    vals = []
    for arg in lst:
        vals.append( float(arg) )
    return vals

def prod(lst):
    """ Computes and returns the product of a list """
    res = 1.
    for x in lst: res *= x
    return res

#### The main program begins here

try:
    float_list = to_floats( sys.argv[1:] )
    print ( prod( float_list ) )
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

Discusion:
* We've created two functions: `to_floats`, to solve the conversion, and `prod`, to compute the product.
* In `to_floats()`, we used the `append` function to add new items to the list.


* In some ways, this code is _worse_ than the implementation we've had before (longer, likely a bit slower), **but it's easier to maintain in the long-run** as it clearly **separates the code that does the work (the algorithms, the "business logic"), from the glue that ties it together**.

## Implementation v3 (with list comprehensions)

In [13]:
show_code_listing("prog11.py")

```python
#!/usr/bin/env python

import sys

def exit_with_msg(msg):
    """ Prints a usage message and exits the program. """
    print("{}\n\nUsage: {} <arg1> <arg2>".format(msg, sys.argv[0]))
    exit(0)

def to_floats(lst):
    """ Returns a list of strings corresponding to a list of floats """
    return [ float(arg) for arg in lst ]

def prod(lst):
    """ Computes and returns the product of a list """
    res = 1.
    for x in lst: res *= x
    return res

#### The main program begins here

try:
    float_list = to_floats( sys.argv[1:] )
    print ( prod( float_list ) )
except ValueError as e:
    exit_with_msg("Error: {}. All arguments must be numbers.".format(e))

```

And that's all there is to it ! (well... maybe 90%... :)) 

## For next time

Read this introduction to Object Oriented Programming:

https://realpython.com/python3-object-oriented-programming/

and work through the examples.